In [1]:
import pyodbc as pb
import pandas as pd
import numpy as np
import cx_Oracle as cx
import re

In [33]:
#Créer la connexion
dsn_tns = cx.makedsn (host='telline.univ-tlse3.fr',port=1521,sid='etupre') 
conn = cx.connect (user = 'bhm0322a', password ='sid2019', dsn = dsn_tns) 
c = conn.cursor ()

## Collection

In [3]:
#Importer la table collection à inserer dans la base
df_collection=pd.read_csv("../data/t_collection.csv")
df_collection.head()

,issn,nom_clt,nb_numeros,nb_document,pre_année_pub,der_année_pub,discpline,editeur
0,2262-3353,Hommes & Migrations,180,4523,1987,2008,"démographie, science politique",Musée de l'Histoire de l'Immigration
1,1950-6694,Revue économique,320,6776,1950,2002,économie,SciencesPo
2,1783-1768,Revue Philosophique de Louvain,441,12292,1894,2015,philosophie,Peeters
3,1957-7966,Population,320,12794,1946,2004,démographie,INED (Institut national d’études démographiques)
4,1777-5418,Revue Européenne des Migrations Internationales,49,814,1985,2001,démographie,MSH Poitiers


In [4]:
#Remplacer les NaN
df_collection.fillna('',inplace=True)

In [5]:
def insertion(cursor,table,value):
    """ Cette fonction,on lui donne un cursor, la table à inserer les données et les données.
        Et elle l'insere
    """
    #Enlever les caractères non prises par oracle
    value=value.encode('ascii', 'ignore').decode('ascii')
    
    #Inserer dans la base
    query="INSERT INTO "+table+" VALUES ("+value+")"
    cursor.execute(query)

In [6]:
def insertion_ss(cursor,table,df):
    """ Cette fonction, on lui donner un cursor, la table à inserer les données et un dataframe.
        Il prend chaque ligne du dataframe et l'insere dans la table
    """
    #Recupere tous les colonnes du dataframe
    cols=list(df.columns)
    
    for ind,row in df.iterrows():
        value=""
        for c in cols:
            if type(row[c])==str:
                #Oracle ne lis pas tous les caractères. Je remplace les caractères évidents
                v=row[c].replace("'"," ")
                v=v.replace("é","e")
                v=v.replace("à","a")
                v=v.replace("è","e")
                v="'"+v+"'"
            else :
                v=str(row[c])
            value+=v+","
        #Inserer dans la table
        insertion(cursor,table,value[:-1])
        
        #Juste pour connaitre l'avance du programme
        if ind%1000==0:
            print(ind)

In [7]:
cursor = conn.cursor()
insertion_ss(cursor,"COLLECTION",df_collection)
conn.commit()
cursor.close()

0


In [8]:
query = "SELECT * FROM COLLECTION"
df = pd.read_sql(query,conn)
df.head()

,ISSN,NOM_COLLECTION,NB_NUMERO,NB_DOCUMENT,ANNEE_CREATION,ANNEE_FIN,DISCIPLINE,EDITEUR
0,1492-1375,Sociologie et societes,100,150,1969,2018,Sociologie,Les Presses de lUniversite de Montreal
1,1773-0201,Cahiers de la Mediterranee,66,801,1970,2000,None,None
2,1777-5809,Reseaux. Communication - Technologie - Societe,126,1876,1983,2000,sciences de l information et de la communicati...,La decouverte
3,1775-3945,"Sante, Societe et Solidarite",18,360,2002,2010,"economie, science politique",Institut de Recherche et Documentation en cono...
4,1777-5795,conomie & prevision,181,1421,1971,2017,"economie, science politique, sociologie",INSEE


## Article

In [9]:
df_article=pd.read_csv("../data/t_article.csv")
df_article.head()

,num_doc,title,annee_publication,issn
0,1,La fuite des cerveaux : le cas de l'ex-URSS,1992,2262-3353
1,2,"Croissance, redistribution et inégalités dans ...",1999,1950-6694
2,3,Que nous devons-nous les uns aux autres ? Le c...,2008,1783-1768
3,4,Chorafas Dimitri N. — La fuite des cerveaux. L...,1970,1957-7966
4,5,Au-delà de la « fuite des cerveaux » : la mobi...,1997,1777-5418


In [10]:
df_article.fillna('',inplace=True)

In [11]:
cursor = conn.cursor()
insertion_ss(cursor,"ARTICLE",df_article)
conn.commit()
cursor.close()

0
1000
2000
3000
4000
5000
6000
7000
8000
9000


In [12]:
query = "SELECT * FROM ARTICLE"
df = pd.read_sql(query,conn)
df.head()

,NUMERO_ARTICLE,TITRE_ARTICLE,ANNEE_PUBLICATION,ISSN
0,81,"Bernard Chavance (sous la dir.), Les incertitu...",2004,2259-6100
1,82,Mobilite internationale etudiante et insertion...,2013,1705-1495
2,83,"Abusharaf R. M., 1997 -Sudanese migration to t...",1999,1961-8603
3,84,La migration armenienne vers Paris et sa regio...,2007,2262-3353
4,85,Laffaire du cerveau. Quelques remarques sur le...,2006,None


## mot_article

In [13]:
df_mot_article=pd.read_csv("../data/t_mot_article.csv")
df_mot_article.head()

,mot,num_article,frequence
0,plus,2,1
1,polit,3,1
2,europ,4,1
3,plus,4,1
4,pay,4,1


In [14]:
cursor = conn.cursor()
insertion_ss(cursor,"article_comporte_mot",df_mot_article)
conn.commit()
cursor.close()

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000


In [15]:
query = "SELECT * FROM article_comporte_mot"
df = pd.read_sql(query,conn)
df.head()

,MOT,NUMERO_ARTICLE,FREQUENCE_ARTICLE
0,tout,221,3
1,europ,222,2
2,pay,222,2
3,europ,223,3
4,pay,223,1


## Auteur

In [16]:
df_auteur=pd.read_csv("../data/t_auteur.csv")
df_auteur.head()

,num,nom,annee_naissance,annee_decès,profession
0,414356,Kortunov Andrei,NaN,NaN,NaN
1,140232,Bouyssou Rachel,NaN,NaN,NaN
2,24789,Docquier Frédéric,NaN,NaN,Docteur en sciences économiques
3,61222,Rapoport Hillel,1963.0,NaN,NaN
4,731465,Chauvier Stéphane,1959.0,NaN,Maître de conférences de philosophie à l'Unive...


In [17]:
df_auteur.fillna('',inplace=True)

In [18]:
cursor = conn.cursor()
insertion_ss(cursor,"auteur",df_auteur)
conn.commit()
cursor.close()

0
1000
2000
3000
4000
5000
6000
7000


In [19]:
query = "SELECT * FROM auteur"
df = pd.read_sql(query,conn)
df

,NUMERO_AUTEUR,NOM_PRENOM,ANNEE_DE_NAISSANCE,ANNEE_DE_DECES,PROFESSION
0,395042,Acselrad Henri,NaN,NaN,Professeur d urbanisme et d amenagement du ter...
1,395043,Mello Cecilia,NaN,NaN,None
2,395066,Vaucher Philippe,NaN,NaN,None
3,247976,Veyne Paul,1930.0,NaN,Historien
4,177537,Song Wu,NaN,NaN,None
...,...,...,...,...,...
7581,396619,Smith Marc H.,1963.0,NaN,Docteur en histoire (Paris
7582,158155,Couturas Claire,1963.0,NaN,None
7583,49821,Bonaventure N.,NaN,NaN,None
7584,49820,Riche N.,NaN,NaN,None


## Redige

In [20]:
df_redige=pd.read_csv("../data/t_redige.csv")
df_redige.head()

,num,num_doc
0,414356,1
1,140232,1
2,24789,2
3,61222,2
4,731465,3


In [23]:
cursor = conn.cursor()
insertion_ss(cursor,"redige",df_redige)
conn.commit()
cursor.close()

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000


In [24]:
query = "SELECT * FROM redige"
df = pd.read_sql(query,conn)
df

,NUMERO_AUTEUR,NUMERO_ARTICLE
0,0004421,443
1,0004422,443
2,59771,444
3,226008,445
4,51235,446
...,...,...
10637,49819,9999
10638,49280,9999
10639,49820,9999
10640,49806,9999


## Nationalite

In [31]:
df_nationalite=pd.read_csv('../data/t_nationalite.csv')
df_nationalite.head()

,num,nom_pays
0,33007,france
1,259994,france
2,249315,france
3,215901,france
4,234171,france


In [34]:
cursor = conn.cursor()
insertion_ss(cursor,"nationalite",df_nationalite)
conn.commit()
cursor.close()

0


In [36]:
query = "SELECT * FROM nationalite"
df = pd.read_sql(query,conn)
df

,NUMERO_AUTEUR,NOM_PAYS
0,163274,afghanistan
1,288736,argentina
2,34139,benin
3,234869,burkina faso
4,197551,burundi
...,...,...
325,401478,reunion
326,174724,romania
327,253971,sri lanka
328,227422,thailand


## Pays_article

In [37]:
df_pays_article=pd.read_csv("../data/t_pays_article.csv")
df_pays_article.head()

,nom_pays,num_article
0,réunion,15
1,poland,36
2,oman,43
3,pakistan,62
4,egypt,66


In [39]:
cursor = conn.cursor()
insertion_ss(cursor,"PAYS_ARTICLE",df_pays_article)
conn.commit()
cursor.close()

0


In [40]:
query = "SELECT * FROM PAYS_ARTICLE"
df = pd.read_sql(query,conn)
df

,NOM_PAYS,NUMERO_ARTICLE
0,reunion,15
1,poland,36
2,oman,43
3,pakistan,62
4,egypt,66
...,...,...
890,egypt,9935
891,finland,9968
892,portugal,9968
893,chad,9972


In [41]:
#Fermer la connexion
conn.close()